# 한국투자증권 REST API

## 1. 서비스 연결(Oauth)

In [1]:
import requests # HTTP 요청을 보낼 때
import json # 특정 객체를 JSON 데이터로 만들어서 쓰기 위해
import yaml

In [5]:
with open("config.yaml", encoding="UTF-8") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    
APP_KEY = cfg["APP_KEY"]
APP_SECRET = cfg["APP_SECRET"]
ACCESS_TOKEN = ""
CANO = cfg["CANO"]
ACNT_PRDT_CD = cfg["ACNT_PRDT_CD"]
URL_BASE = cfg["URL_BASE"]

In [3]:
headers = {
    "content-type": "application/json"
    }
body = {
    "grant_type": "client_credentials",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET
    }
PATH = "oauth2/tokenP"

URL = f"{URL_BASE}/{PATH}"
print(URL)

https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [4]:
res = requests.post(URL, headers=headers, data=json.dumps(body))

In [5]:
# 보안인증키 : access_token
ACCESS_TOKEN = res.json()["access_token"]

## 2. 주식현재가 조회
### 현재가 : FHKST01010100
### 일자별 : FHKST01010400

In [6]:
# 현재가 조회
PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
URL = f"{URL_BASE}/{PATH}"
print(URL)

headers = {
    "content-type": "application/json",
    "authorization": f"Bearer {ACCESS_TOKEN}",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET,
    "tr_id": "FHKST01010100"
    }

params = {
    "fid_cond_mrkt_div_code": "J",
    "fid_input_iscd": "005930"
}

https://openapivts.koreainvestment.com:29443/uapi/domestic-stock/v1/quotations/inquire-price


In [7]:
res = requests.get(URL, headers=headers, params=params)

In [8]:
res.json()["output"]["stck_prpr"]

'59700'

In [9]:
# 일자별 조회
PATH = "uapi/domestic-stock/v1/quotations/inquire-daily-price"
URL = f"{URL_BASE}/{PATH}"
headers["tr_id"] = "FHKST01010400"

params = {
    "fid_cond_mrkt_div_code": "J",
    "fid_input_iscd": "005930",
    "fid_org_adj_prc": "1",
    "fid_period_div_code": "D"
}

In [10]:
res = requests.get(URL, headers=headers, params=params)

In [11]:
len(res.json()["output"])

30

In [12]:
res.json()["output"][0]

{'stck_bsop_date': '20220825',
 'stck_oprc': '59200',
 'stck_hgpr': '59700',
 'stck_lwpr': '59000',
 'stck_clpr': '59700',
 'acml_vol': '5767902',
 'prdy_vrss_vol_rate': '-35.11',
 'prdy_vrss': '700',
 'prdy_vrss_sign': '2',
 'prdy_ctrt': '1.19',
 'hts_frgn_ehrt': '49.81',
 'frgn_ntby_qty': '207759',
 'flng_cls_code': '00',
 'acml_prtt_rate': '1.00'}

## 3. 해쉬키 발급
### 보안을 위한 요소로 사용자가 보낸 요청 값을 중간에 탈취하여 변조하지 못하도록 하는데 사용, POST로 보내는 요청(주문)

In [13]:
datas = {
    "CANO": "00000000",
    "ACNT_PRDT_CD": "01",
    "OVRS_EXCG_CD": "SHAA",
    "PDNO": "00001",
    "ORD_QTY": "500",
    "OVRS_ORD_UNPR": "52.62",
    "ORD_SVR_DVSN_CD": "0"
}

headers = {
    "content-type": "application/json",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET
}

PATH = "uapi/hashkey"
URL = f"{URL_BASE}/{PATH}"
print(URL)

https://openapivts.koreainvestment.com:29443/uapi/hashkey


In [14]:
res = requests.post(URL, headers=headers, data=json.dumps(datas))

In [15]:
hashkey = res.json()["HASH"]

## 4-1. 매수 주문
### tr_id : VTTC0802U

In [16]:
def hashKey(datas):
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
        'content-Type' : 'application/json',
        'appKey' : APP_KEY,
        'appSecret' : APP_SECRET,
        }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]

    return hashkey

In [17]:
PATH = "uapi/domestic-stock/v1/trading/order-cash"
URL = f"{URL_BASE}/{PATH}"

data = {
    "CANO": CANO,
    "ACNT_PRDT_CD": ACNT_PRDT_CD,
    "PDNO": "005930", # 종목
    "ORD_DVSN": "01", # 시장가
    "ORD_QTY": "10", # 수량
    "ORD_UNPR": "0", # 시장가 주문
    }

headers = {
    "content-type": "application/json", 
    "authorization": f"Bearer {ACCESS_TOKEN}",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET,
    "tr_id": "VTTC0802U",
    "custtype": "P",
    "hashkey" : hashKey(data)
    }

In [18]:
res = requests.post(URL, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40910000', 'msg1': '모의투자 주문이 불가한 계좌입니다.'}

In [ ]:
KRX_FWDG_ORD_ORGNO = res.json()["output"]["KRX_FWDG_ORD_ORGNO"] # 한국거래소전송주문조직번호
ODNO = res.json()["output"]["ODNO"] # 주문번호

## 4-2. 매도 주문

In [ ]:
PATH = "uapi/domestic-stock/v1/trading/order-cash"
URL = f"{URL_BASE}/{PATH}"

data = {
    "CANO": CANO,
    "ACNT_PRDT_CD": ACNT_PRDT_CD,
    "PDNO": "005930", # 종목
    "ORD_DVSN": "01", # 시장가
    "ORD_QTY": "10", # 수량
    "ORD_UNPR": "0", # 시장가 주문
    }

headers = {
    "content-type": "application/json", 
    "authorization": f"Bearer {ACCESS_TOKEN}",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET,
    "tr_id": "VTTC0801U",
    "custtype": "P",
    "hashkey" : hashKey(data)
    }

res = requests.post(URL, headers=headers, data=json.dumps(data))
res.json()